<a href="https://colab.research.google.com/github/mayhd3/NSF-REU-2021/blob/main/prepdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os

if not os.path.exists('Irish_dataset.csv'):
  !gdown --id 1OFQo6CmBPwn3FRB5EsRm9t6wba_5Lcr9


In [3]:
from __future__ import print_function
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import ADASYN
# from keras.models import Sequential
# from keras.layers import Dense, LSTM, Merge, GRU
# from keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.utils import shuffle
import random
import math
import copy
import sys
import pickle
import os
import csv

i = 0 # counter for customers
a_dr = 0 # detection rate initialization
a_fr = 0 # false rejection initialization 
a_fa = 0 # false acceptance initialization
fIn = "Irish_dataset.csv" 
# fOt1 = sys.argv[2]
np.random.seed(1234) # Set the random seed for numpy
# flog1 = open(fOt1,'w')
scores = list()


# creat six types of attacks
def create_attacks(d):
	alpha1 = random.uniform(0.1,0.6)
	at1 = np.dot(alpha1,d) #1st attack type	
	alpha2 = np.random.uniform(low=0.1, high=0.6, size=(536,48))
	at3 = np.multiply(alpha2,d) # 3rd attack type
	avg = np.mean(d,axis=1) # customer daily average consumption
	avg_m = np.diag(avg) # in diagonal matrix for multiplication  
	alpha3 = np.random.uniform(low=0.1, high=0.6, size=(536,48))
	at4 = np.dot(avg_m,alpha3) # 4th attack type
	alpha4 = np.ones((536,48))
	at5 = np.dot(avg_m,alpha4) # 5th attack type	
	return at1,at3,at4,at5

# Split the data to train and test
def split_train_test(X_res,Y_res):
	ind1 = np.where(Y_res == 0)[0]
	XH = X_res[ind1,:] # honest samples
	YH = np.zeros(ind1.shape[0])
	ind_H = int(math.ceil(ind1.shape[0]*4/5))
	XH_TR = XH[0:ind_H,:]
	YH_TR = YH[0:ind_H]
	XH_TST = XH[ind_H:,:]
	YH_TST = YH[ind_H:] 
	XC = np.concatenate((at1,at3,at4,at5), axis=0) # all attacks
	YC = np.ones(XC.shape[0]) 
	ind_C = int(math.ceil(536*4/5))
	XC_TR = np.concatenate((at1[0:ind_C,:],at3[0:ind_C,:],at4[0:ind_C,:],at5[0:ind_C,:]), axis=0) # training cheating samples
	YC_TR = YC[0:XC_TR.shape[0]]
	XC_TST = np.concatenate((at1[ind_C:,:],at3[ind_C:,:],at4[ind_C:,:],at5[ind_C:,:]), axis=0) # test cheating samples
	YC_TST = YC[XC_TR.shape[0]:]     
	XTR = np.concatenate((XH_TR,XC_TR), axis=0)  
	YTR = np.concatenate((YH_TR,YC_TR), axis=0) 
	ind = list(range(XTR.shape[0]))  #random shuffling the training data
	np.random.shuffle(ind)
	XTR = XTR[ind]
	YTR = YTR[ind]
	XTST = np.concatenate((XH_TST,XC_TST), axis=0)   
	YTST = np.concatenate((YH_TST,YC_TST), axis=0)   
	return XTR,YTR,XTST,YTST

In [ ]:

print ("Loading Data ...")
D = np.array(list(csv.reader(open(fIn, "rt"), delimiter=","))).astype("float")
D = np.loadtxt(fIn, delimiter=",")
D = np.transpose(D[1:,:])
x_tr = open('Xtr_all_200.csv','ab')
y_tr = open('Ytr_all_200.csv','ab')
x_tst = open('Xtst_all_200.csv','ab')
y_tst = open('Ytst_all_200.csv','ab')

for d in D: # for each user
  if i==200:
    break
  i+=1
  print(i)
  file_name = 'Users/'+str(i)+'.pkl'
  #if training data for that user exist load it (balanced data set for each user has both honest and malicious samples) 
  if os.path.exists(file_name):
    [XTR,YTR,XTST,YTST] = pickle.load(open(file_name, 'rb'))
  else:
    d = d.reshape(536,48) # customer data arranged in matrix: rows = days, columns = time
      ############################################################## 
      # Creating attacks
    at1, at3, at4, at5 = create_attacks(d)
      ##############################################################    
      # Over-sampling
    X = np.concatenate((d,at1,at3,at4,at5), axis=0) # all data = honest + attacks
    y1 = np.zeros(536)
    y2 = np.ones(536*6)
    Y = np.concatenate((y1,y2), axis=0)
    ada = ADASYN(ratio='minority',random_state=42)
    X_res, Y_res = ada.fit_sample(X,Y) # over-sampled data
      
      ##############################################################    
      # Split data to train 2/3 and test 1/3
    XTR,YTR,XTST,YTST = split_train_test(X_res,Y_res) 
    #pickle.dump([XTR,YTR,XTST,YTST], open(file_name, 'wb'))
      ############################################################## 				
  np.savetxt(x_tr, XTR, delimiter=",")
  np.savetxt(y_tr, YTR, delimiter=",")
  np.savetxt(x_tst, XTST, delimiter=",")
  np.savetxt(y_tst, YTST, delimiter=",")

Loading Data ...
